<a href="https://colab.research.google.com/github/FazleRabbbiferdaus172/machine_learning/blob/master/Adaline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Adaline**
##Adaptive Linear Neuron
###Updated based on a linear activation function 

In [ ]:
import numpy as np

class AdalineGD(object):
    """Perceptron classifier.

    Parameters
    ------------
    eta : float
        Learning rate (between 0.0 and 1.0)
    n_iter : int
        Passes over the training dataset.
    random_state : int
        Random number generator seed for random weight initialization.

    Attributes
    -----------

    w_ : 1d-array
        Weights after fitting.
    cost_ : list
        Sum-of-squres cost function value in each epoch
    """

    def __init__(self, eta = 0.01, n_iter = 50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """Fit traning data.
        
        Parameters
        -----------
        X : {array-like}, shape = [n_samples, n_features]
            Traning vectors, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values.
        
        Returns
        ---------
        self : object

        """

        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale= 0.01, size=1 + X.shape[1])

        self.cost_ = []

        for _ in range(self.n_iter):
            net_input = self.net_input(X)
            output = self.activation(net_input)
            errors = (y - output)
            self.w_[1:] += self.eta * X.T.dot(errors) #calculates the gradient based on the whole training dataset  for the weights 1 to m
            self.w_[0] += self.eta * errors.sum()   #calculates the gradient based on the whole training dataset for bias
            cost = (error**2).sum() / 2.0
            self.cost_.append(cost)
        return self

    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def activation(self, X):
        """Compute liner activation"""    
        return X    # Activation method has no effect in the code since it is simply an identity function

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, -1)